*CELDA 1: Instalación de dependencias*

In [ ]:
# --- NUEVA CELDA 1 (Versión Definitiva) ---
# Instalamos TODO el ecosistema en una sola línea para forzar
# a pip a resolver todas las dependencias juntas y evitar conflictos.

!pip install -qU langchain langgraph langsmith langchain-google-genai \
                 chromadb pypdf notion-client \
                 sentence-transformers langchain-community

print("--- ¡Instalación completa (Modo Unificado)! ---")

--- ¡Instalación completa (Modo Unificado)! ---


Celda 2:*CARGAR SECRETOS*

In [ ]:
# --- NUEVA CELDA 2 (Carga de Secretos) ---
from google.colab import userdata
import os

print("Cargando secretos...")

# 1. Google (Para el LLM)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# 2. LangSmith (Para Observabilidad - Req. 5)
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true" # ¡Activa LangSmith!
os.environ["LANGCHAIN_PROJECT"] = "Proyecto Tutor IA" # (Nombre de tu proyecto en LangSmith)

# 3. Notion (Para Persistencia - Req. 4)
os.environ["NOTION_API_KEY"] = userdata.get('NOTION_API_KEY')
os.environ["NOTION_DATABASE_ID"] = userdata.get('NOTION_DATABASE_ID')

print("¡Secretos cargados en el entorno!")

Cargando secretos...
¡Secretos cargados en el entorno!


**Base de Conocimiento (RAG)** CELDA 3

In [ ]:
# --- NUEVA CELDA 3 (Corregida con Embeddings Locales) ---

import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
# ¡CAMBIO 1! Importamos los embeddings locales de Hugging Face
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

print("Iniciando Fase 1 (RAG) con ChromaDB...")

# --- 1. Cargar la API Key de Google ---
try:
    if not os.environ.get("GOOGLE_API_KEY"):
         os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("Google API Key cargada.")
except Exception as e:
    print(f"Error al cargar GOOGLE_API_KEY: {e}.")

# --- 2. Definir Modelos (LLM y Embeddings) ---
try:
    # El LLM (Chat) sigue usando Google
    llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro", temperature=0)
    print("Modelo de Google (LLM) cargado.")

    # ¡CAMBIO 2! Usamos un modelo local y gratuito de Hugging Face
    # Esto evita el error de quota de Google.
    # (La primera vez que se ejecute, descargará el modelo. Tarda ~1 min)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    print("Modelos de Embeddings (Local/Gratuito) cargado.")

except Exception as e:
    print(f"Error al cargar modelos: {e}.")


# --- 3. Cargar Documentos de la Materia (PDFs y TXTs) ---
documentos_path = "/content/documentos_rag"

if not os.path.exists(documentos_path) or not os.listdir(documentos_path):
    print(f"Error: La carpeta '{documentos_path}' no existe o está vacía.")
    print("Por favor, sube tus documentos (PDFs, .txt) a esa carpeta en Colab.")
else:
    print(f"Cargando documentos desde: {documentos_path}")

    # Cargar archivos PDF
    print("Buscando archivos .pdf...")
    loader_pdf = DirectoryLoader(documentos_path, glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True)
    documentos_pdf = loader_pdf.load()
    print(f"Se cargaron {len(documentos_pdf)} documentos PDF.")

    # Cargar archivos TXT
    print("Buscando archivos .txt...")
    loader_txt = DirectoryLoader(documentos_path, glob="**/*.txt", loader_cls=TextLoader, show_progress=True, loader_kwargs={"encoding": "utf-8"})
    documentos_txt = loader_txt.load()
    print(f"Se cargaron {len(documentos_txt)} documentos TXT.")

    documentos = documentos_pdf + documentos_txt

    if not documentos:
        print("¡Error! No se cargó ningún documento.")
    else:
        print(f"Total de documentos cargados (PDF + TXT): {len(documentos)}.")

        # --- 4. Dividir (Split) ---
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        docs_divididos = text_splitter.split_documents(documentos)
        print(f"Los documentos se dividieron en {len(docs_divididos)} trozos (chunks).")

        # --- 5. Almacenar (Store en ChromaDB) ---
        print("Creando la base de datos vectorial (ChromaDB)...")
        # Esta línea ahora usará los 'embeddings' locales (¡no usará la API de Google!)
        db = Chroma.from_documents(docs_divididos, embeddings)
        print("Base de datos vectorial creada.")

        # --- 6. Crear el Retriever (El "Buscador") ---
        retriever = db.as_retriever(search_kwargs={"k": 5})
        print("¡Retriever (Buscador) listo!")

        # --- 7. Prueba rápida ---
        print("\n--- Prueba del Retriever ---")
        pregunta_prueba = "Hazme un resumen de uno de los temas de los documentos" # Cambia esto por una pregunta real
        try:
            contexto = retriever.invoke(pregunta_prueba)
            print(f"Resultados de la búsqueda para: '{pregunta_prueba}'")
            if contexto:
                print(contexto[0].page_content[:500] + "...")
            else:
                print("No se encontraron resultados relevantes.")
        except Exception as e:
            print(f"Error en la prueba del retriever: {e}")

Iniciando Fase 1 (RAG) con ChromaDB...
Google API Key cargada.
Modelo de Google (LLM) cargado.
Modelos de Embeddings (Local/Gratuito) cargado.
Cargando documentos desde: /content/documentos_rag
Buscando archivos .pdf...


0it [00:00, ?it/s]


Se cargaron 0 documentos PDF.
Buscando archivos .txt...


100%|██████████| 5/5 [00:00<00:00, 2064.53it/s]

Se cargaron 5 documentos TXT.
Total de documentos cargados (PDF + TXT): 5.
Los documentos se dividieron en 56 trozos (chunks).
Creando la base de datos vectorial (ChromaDB)...


Base de datos vectorial creada.
¡Retriever (Buscador) listo!

--- Prueba del Retriever ---
Resultados de la búsqueda para: 'Hazme un resumen de uno de los temas de los documentos'
*Contenido: El informe es detallado y técnico
*Extensión: El informe de auditoría es extenso.
Está dirigido a auditores, técnicos y personal de TI. 
Informe Ejecutivo: Es un resumen de alto nivel del informe completo de la auditoría, diseñado para ser breve y ser leído por la alta dirección o personas que no necesitan conocer todos los detalles técnicos. 
*Audiencia: alta dirección y no técnicos.
*Contenido: resumen de estratégico y de alto nivel
*Extensión: breve
El informe debe tener las sigui...


Celda 4: **Definición de Herramientas**

In [ ]:
# --- NUEVA CELDA 4 (con la importación DEFINITIVA Y CORRECTA) ---

import os
from langchain_core.tools import tool, create_retriever_tool
from notion_client import Client

print("Iniciando Fase 2: Definición de Herramientas...")

# --- 1. Herramienta 1: Notion (Persistencia) ---

NOTION_API_KEY = os.environ.get('NOTION_API_KEY')
NOTION_DATABASE_ID = os.environ.get('NOTION_DATABASE_ID')

try:
    notion = Client(auth=NOTION_API_KEY)
    print("Cliente de Notion conectado.")
except Exception as e:
    print(f"Error al conectar con Notion: {e}")
    print("Verifica tu NOTION_API_KEY y NOTION_DATABASE_ID en los Secretos (Celda 2).")

@tool
def guardar_resumen_en_notion(titulo: str, contenido: str):
    """
    Usa esta herramienta para guardar un resumen o una explicación generada
    en la base de datos de Notion del estudiante.
    Recibe un 'titulo' para el resumen y el 'contenido' del mismo.
    """

    if not NOTION_API_KEY or not NOTION_DATABASE_ID:
        return "Error: Las variables de entorno de Notion no están configuradas."

    print(f"\n🤖 API NOTION: Intentando guardar '{titulo}'...")
    try:
        nueva_pagina = {
            "parent": {"database_id": NOTION_DATABASE_ID},
            "properties": {
                "Nombre": {
                    "title": [
                        {"text": {"content": titulo}}
                    ]
                }
            },
            "children": [
                {
                    "object": "block",
                    "type": "heading_2",
                    "heading_2": {
                        "rich_text": [{"type": "text", "text": {"content": "Resumen Generado"}}]
                    }
                },
                {
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [
                            {"type": "text", "text": {"content": contenido}}
                        ]
                    }
                }
            ]
        }

        notion.pages.create(**nueva_pagina)
        print(f"🤖 API NOTION: ¡Éxito! Resumen guardado.")
        return f"Éxito: El resumen titulado '{titulo}' fue guardado en Notion."

    except Exception as e:
        print(f"Error al guardar en Notion: {e}")
        return f"Error al intentar guardar en Notion: {e}"

# --- 2. Herramienta 2: Off-Topic ---
@tool
def off_topic_tool():
    """
    Se activa cuando el usuario pregunta algo no relacionado con la materia,
    los apuntes, o generar resúmenes.
    """
    return "Disculpa, como tu tutor virtual, solo puedo ayudarte con el contenido de la materia. ¿Tienes alguna pregunta sobre los apuntes?"

# --- 3. Herramienta 3: RAG (Buscador de Apuntes) ---
# (El 'retriever' fue creado en la Celda 3)
consultar_apuntes_tool = create_retriever_tool(
    retriever, # Esta variable DEBE existir de la Celda 3
    "consultar_apuntes_materia",
    "Busca y recupera información de los apuntes de la materia. Úsalo para responder cualquier pregunta sobre el contenido académico, definiciones, teoremas o ejemplos."
)

# --- 4. Lista final de herramientas ---
tools = [consultar_apuntes_tool, guardar_resumen_en_notion, off_topic_tool]

print("\n--- ¡Herramientas definidas y listas! ---")
for t in tools:
    print(f"- {t.name}")

# --- 5. Prueba rápida de Notion ---
print("\n--- Prueba de la Herramienta Notion ---")
try:
    # (Descomenta la siguiente línea para probar guardar algo ahora mismo)
    # print(guardar_resumen_en_notion(titulo="Prueba de Celda 4", contenido="Si ves esto, la herramienta de Notion funciona."))
    print("Herramienta 'guardar_resumen_en_notion' definida.")
    print("¡Fase 2 (Tools) completada!")
except Exception as e:
    print(f"Error en la prueba de Notion: {e}")

Iniciando Fase 2: Definición de Herramientas...
Cliente de Notion conectado.

--- ¡Herramientas definidas y listas! ---
- consultar_apuntes_materia
- guardar_resumen_en_notion
- off_topic_tool

--- Prueba de la Herramienta Notion ---
Herramienta 'guardar_resumen_en_notion' definida.
¡Fase 2 (Tools) completada!


Celda 5: **CONSTRUIR EL AGENTE**

In [ ]:

from typing import Sequence, Annotated, TypedDict, Literal
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

print("Iniciando Fase 3: Construcción del Grafo...")

# --- 1. Vincular Herramientas al LLM ---
# El 'llm' ya existe desde la Celda 3
# Las 'tools' ya existen desde la Celda 4
llm_with_tools = llm.bind_tools(tools)

# --- 2. Definir el Estado (AgentState) ---
# Esta es la plantilla exacta de 'Bruno el Mozo'
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- 3. Definir los Nodos del Grafo ---

# NODO 1: El Agente (El "Cerebro" que decide)
def agent_node(state: AgentState):
    """Invoca al LLM (Tutor) para que decida el siguiente paso."""

    # ¡ESTE ES EL NUEVO PROMPT! (La "personalidad" del Tutor)
    system_prompt = """
    Eres un "Agente Tutor" de IA. Tu propósito es ayudar a un estudiante a estudiar
    el contenido de sus apuntes. Eres amable, pedagógico y preciso.

    Instrucciones:
    1.  Saluda al estudiante y preséntate.
    2.  Utiliza la herramienta `consultar_apuntes_materia` para responder CUALQUIER pregunta académica (ej. "¿Qué es un Teorema?", "¿Explícame X concepto?").
    3.  Basa tus respuestas ÚNICAMENTE en la información de los apuntes. Si la información no está, indica que no la encontraste en el material.
    4.  Si el usuario te pide "haz un resumen de X" o "guarda esta explicación", genera el contenido (usando `consultar_apuntes_materia` si es necesario) y LUEGO usa la herramienta `guardar_resumen_en_notion` para persistirlo.
    5.  Si la pregunta no tiene NADA que ver con la materia, DEBES usar la herramienta `off_topic_tool`.
    """

    # Añadimos el prompt del sistema al historial de mensajes
    messages = [SystemMessage(content=system_prompt)] + state["messages"]

    # Invocamos al LLM con las herramientas
    response = llm_with_tools.invoke(messages)

    # Devolvemos la respuesta para añadirla al estado
    return {"messages": [response]}

# NODO 2: Las Herramientas (Las "Manos" que actúan)
# Este nodo es una función especial de LangGraph que ejecuta las herramientas
tools_node = ToolNode(tools)

# --- 4. Definir las Transiciones (Edges) ---

def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    """Determina si se debe llamar a una herramienta o si el flujo ha terminado."""
    # Revisa el último mensaje. Si tiene una 'tool_call', debe ir al nodo 'tools'
    if state["messages"][-1].tool_calls:
        return "tools"
    # Si no, la conversación termina y el agente responde
    return "__end__"

# --- 5. Construir y Compilar el Grafo ---

graph_builder = StateGraph(AgentState)

# Añadimos los dos nodos
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("tools", tools_node)

# El punto de entrada es el 'agent'
graph_builder.set_entry_point("agent")

# Añadimos las transiciones condicionales
graph_builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",  # Si "should_continue" devuelve "tools", va al nodo "tools"
        "__end__": END     # Si devuelve "__end__", termina
    }
)

# Después de que el nodo 'tools' termina, siempre vuelve al 'agent'
graph_builder.add_edge("tools", "agent")

# ¡Compilamos nuestro agente!
tutor_agent = graph_builder.compile()

print("\n🧠 ¡Grafo del Agente Tutor construido y compilado!")
print("¡Fase 3 (LangGraph) completada!")

Iniciando Fase 3: Construcción del Grafo...

🧠 ¡Grafo del Agente Tutor construido y compilado!
¡Fase 3 (LangGraph) completada!


In [ ]:
# Esta línea le pide al agente que describa su propia arquitectura
# en el lenguaje de diagramas "Mermaid".
print(tutor_agent.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	agent(agent)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> agent;
	agent -.-> __end__;
	agent -.-> tools;
	tools --> agent;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



**INTERACTUAR CON EL AGENTE**

In [ ]:
from langchain_core.messages import HumanMessage
# Guardamos el historial en una lista
conversation_history = []

print("\n" + "="*50)
print("       📚 BIENVENIDO AL AGENTE TUTOR DE IA 📚")
print("="*50)
print("\nTu tutor virtual está listo para ayudarte a estudiar.")
print(" (Escribe 'salir' para terminar)")

# Bucle infinito para chatear
while True:
    query = input("\n👤 Estudiante: ")
    if query.lower() in ["exit", "quit", "salir"]:
        print("\n🤖 Tutor: ¡Excelente sesión de estudio! ¡Hasta la próxima!")
        break

    # 1. Añadimos el mensaje del usuario al historial
    conversation_history.append(HumanMessage(content=query))

    # 2. Invocamos el agente con TODO el historial
    # ¡LangSmith registrará esto automáticamente!
    result = tutor_agent.invoke({"messages": conversation_history})

    # 3. Actualizamos el historial con la respuesta del agente
    conversation_history = result["messages"]

    # 4. Mostramos la respuesta final al usuario
    final_response = conversation_history[-1].content
    print(f"\n🤖 Tutor: {final_response}")


       📚 BIENVENIDO AL AGENTE TUTOR DE IA 📚

Tu tutor virtual está listo para ayudarte a estudiar.
 (Escribe 'salir' para terminar)

👤 Estudiante: Que es un riesgo


NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
!pip freeze > requirements.txt